# Celovitost in zgoščevanje

Cilji laboratorijske vaje so sledeči:
- Spoznati uporabo izbranih mehanizmov zagotavljanja celovitosti v programskih jezikih Python in Java;
- Spoznati uporabo zgoščevanja v programskih jezikih Python in Java;
- Spoznati simulator komunikacijskega kanala v Javi;
- Spoznati mehanizem deduplikacije.

Z uporabo knjižnice [cryptography](https://cryptography.io/en/latest) implementirajte spodnje naloge.

## Naloga 1
Implementirajte metodi `cmac_sign(key, msg)` in `cmac_verify(key, msg, tag)` za CMAC. Za bločno šifro uporabite AES.

In [1]:
import os

In [2]:
from cryptography.hazmat.primitives.cmac import CMAC
from cryptography.hazmat.primitives.ciphers import algorithms
def cmac_sign(key: bytes, msg: bytes) -> bytes:
    cmac = CMAC(algorithms.AES(key))
    cmac.update(msg)
    return cmac.finalize()


In [3]:
def test_cmac_sign():
    key = bytes.fromhex("e3e62dde91fc699fc6195b9387b97b51")
    msg = b"Hello World!"
    tag = bytes.fromhex("cebba0274a6e91a2918e801765b5c692")
    assert cmac_sign(key, msg) == tag
    
test_cmac_sign()

In [4]:
def cmac_verify(key: bytes, msg: bytes, tag: bytes) -> bool:
    if cmac_sign(key, msg) == tag:
        return True
    else:
        return False

In [5]:
def test_cmac_verify():
    key = bytes.fromhex("e3e62dde91fc699fc6195b9387b97b51")
    msg = b"Hello World!"
    tag = bytes.fromhex("cebba0274a6e91a2918e801765b5c692")
    
    assert cmac_verify(key, msg, tag)
    assert not cmac_verify(key, msg, bytes.fromhex("cebba0274a6e91a2918e801765b5c690"))
    assert not cmac_verify(bytes.fromhex("e3e62dde91fc699fc6195b9387b97b50"), msg, tag)
    assert not cmac_verify(key, msg + b"a", tag)
    
test_cmac_verify()

## Naloga 2

Implementirajte metodi `hmac_sign(key, msg)` in `hmac_verify(key, msg, tag)` za HMAC. Kot zgoščevalno funkcijo uporabite SHA-256.

In [6]:
from cryptography.hazmat.primitives.hmac import HMAC, hashes
def hmac_sign(key: bytes, msg: bytes) -> bytes:
    hmac = HMAC(key, hashes.SHA256())
    hmac.update(msg)
    return hmac.finalize()
    

In [7]:
def test_hmac_sign():
    key = bytes.fromhex("9008c42e77571b6615ad7058dc846b44")
    msg = b"Hello World!"
    tag = bytes.fromhex("8958bcd856b90d35ec4fe83e5cfb01224ceaf0040787f0535d7c97b383765d4f")
    
    assert hmac_sign(key, msg) == tag
    
test_hmac_sign()    

In [8]:
def hmac_verify(key: bytes, msg: bytes, tag: bytes) -> bool:
    if hmac_sign(key, msg) == tag:
        return True
    else:
        return False
    

In [9]:
def test_hmac_verify():
    key = bytes.fromhex("ac77bc30018022de3ecee0c3226c7c7d")
    msg = b"Hello World!"
    tag = bytes.fromhex("a2276626e56ae4138f865f0998a4abe9940dbf564a8882d85af951e299811fbb")
    
    assert hmac_verify(key, msg, tag)
    assert not hmac_verify(key, msg, bytes.fromhex("a2276626e56ae4138f865f0998a4abe9940dbf564a8882d85af951e299811fb0"))
    assert not hmac_verify(bytes.fromhex("ac77bc30018022de3ecee0c3226c7c70"), msg, tag)
    assert not hmac_verify(key, msg + b"a", tag)
    
test_hmac_verify()

## Naloga 3

Z vam ljubim urejevalnikom besedil, ali še boljše IDE, odprite projekt **java-integrity**; [sam uporabljam Intellij IDEA Community Edition.](https://www.jetbrains.com/idea/download/download-thanks.html) Če uporabljate IDE, potem odprite datoteko `pom.xml`. Tako boste povečali možnost, da se projekt naloži pravilno.

### Naloga 3.1

Preglejte kodo v primeru `MessageDigestExample.java` in se prepričajte, da razumete njeno delovanje.

Spremenite primer, da boste uporabili zgoščevalno funkcijo SHA-3 z izvlečki dolžine 256 bitov.

### Naloga 3.2

Preglejte kodo v primeru `HMACExample.java` in se prepričajte, da razumete njeno delovanje.

Spremenite primer, da boste uporabili zgoščevalno funkcijo SHA-3 z izvlečki dolžine 512 bitov.

## Naloga 4

Poglejte primer komunikacije v datoteki `CommunicationExampleHMAC.java`. 

To je primer simulacije komunikacijskega kanala med Ano in Borom. Njuno delovanje je podano znotraj metode `task()`. Obe metodi se izvajata sočasno. Tako lahko simuliramo komuniciranje med dejanskimi agenti.

Za pošiljanje in sprejemanje sporočil uporabite metodi `void send(byte[])` in `byte[] receive(String)`. 

Poženite datoteko in poglejte izpis. 

Nato primer spremenite.
- Globalno v metodi `main()` določite simetrični ključ.
- V agentu `Ana` instanciirajte algoritem MAC, uporabite simetrični ključ in izračunajte overitveno kodo sporočila tj. značko.
- Poleg samega sporočila sedaj `Bor`u pošljite še značko. Značko lahko pošljete kot novo sporočilo tj. da dvakrat pokličete metodo `send(byte[])`: enkrat za pošiljanje značke, drugič za pošiljanje sporočila.
- Podobno naj sedaj `Bor` prejme omenjeni sporočili, instanciira algoritem MAC, uporabi simetrični ključ in sam izračuna značko iz prejetega sporočila.
- Izračunano značko naj zatem **varno** primerja s prejeto značko.
- Če se ne ujemata, se naj program zaključi.
- Če se ujemata, Bor izpiše prejeto sporočilo.

## Naloga 5

Zaščita datotek pri pisanju na disk.

Preučite delovanje datotek `WriteToFileHMAC.java` in `ReadFromFileHMAC.java`. 

### Naloga 5.1: Shranjevanje

Datoteka `WriteToFileHMAC.java` je primer, kako podatek zapišemo na disk in na njegov začetek shranimo overitveno kodo sporočila. Bodite pozorni na podrobnosti, kot so:
- način shranjevanja datotek v Javi,
- način, kako se polja bajtov v Javi konkatenirajo,
- način, kako iz instance tipa `Key` preberemo bajte ključa.

### Naloga 5.2: Branje

Datoteka `ReadFromFileHMAC.java` je primer, kako podatek preberemo z diska in preverimo njegovo celovitost. Dodatno vidimo še, kako lahko iz poljubnih bajtov ustvarimo objekt tipa `Key`. Bodite pozorni na podrobnosti, kot so:
- način branja datotek v Javi,
- način, kako iz polja bajtov naredimo ključ tipa `Key`,
- način, kako polje bajtov razdelimo v dva dela: del, ki predstavlja značko in del, ki predstavlja sporočilo.

## Naloga 6

V nalogi 5.2 smo v programskem jeziku Java ustvarili sporočilo, iz njega izračunali overitveno kodo sporočila ter oboje zapisali v datoteko. Uporabili smo algoritem HMAC-SHA256.

- Ključ smo shranili v datoteko `../data/msg.bin.key`.
- Konkatenacijo značke in sporočila pa v `../data/msg.bin`.

V programskem jeziku Python implementirajte funkcijo `open_java_msg()`, ki naj nudi enako funkcionalnost kot naloga 5.2 v Javi: 
- Preberite ključ, značko in sporočilo.
- Preverite, ali je značka veljavna (tj. prepričajte se, da se sporočilo ni spremenilo), in le v tem primeru izpišite sporočilo.

In [ ]:
import hmac
from pathlib import Path

# ../data/assignment7.pkl in ../data/ sem zamenjal z moj path saj sose mi datoteke rahlo premaknile.

def open_java_msg():
    with open(Path("moj_path\\msg.bin.key"), "rb") as f:
        keyBytes = f.read()
        
    with open(Path("moj_path\\msg.bin"), "rb") as f:
        data = f.read()
    
    r_tag = data[:32]
    message = data[32:]
    
    c_tag = hmac.new(keyBytes, message, "sha256").digest()
    if (hmac.compare_digest(r_tag, c_tag)):
        print("Data appears to be valid\n:")
        print(message.decode("utf-8"))
    else:
        print("Read invalid data, aborting!")
    
        
open_java_msg()

Data appears to be valid
:
Sporočilo, ki bo zapisano na disk in zavarovano pred spremembami.


## Naloga 7

Na zadnje si poglejmo primer uporabe zgoščevanja, ki ni neposredno povezan z varnostjo temveč s prihranki prostora na podatkovnih nosilcih. Postopek je znan pod imenom deduplikacija.

Predstavljajte si, da upravjlate z oblačno storitvijo gostovanja datotek kot je npr. Dropbox. Na vaše strežnike veliko odjemalcev nalaga datoteke. Opazite, da je veliko datotek podvojenih. Tako ni nobene potrebe, da če npr. 10 uporabnikov naloži isto datoteko, da na strežniku hranite 10 kopij: dovolj je le ena.

Težava je v tem, da vsak uporabnik poimenuje datoteko po svoje. Zato se odločite, da boste za vsako datoteko izračunalni njeno zgoščeno vrednost in vse te zgostitve hranili v slovarju datotek.

Posamezen ključ v slovarju predstavljala zgoščeno vrednost, pripadajoča vrednost v slovarju pa naj bo **množica** (angl. set) datotek, katerih vsebina je omenjena zgoščena vrednost. 

Implementirajte funkcijo `deduplication(filepath)`, ki na podani lokaciji na datotečnem sistemu prebere vse datoteke s končnico `.bin` in pri vsaki izračuna zgoščeno vrednost njene vsebine z zgoščevalno funkcijo SHA-256.

Zgoščena vrednost se zapiše v slovar kot ključ, kot pripadajoča vrednost pa naj bo množica imen datotek z enako vsebino. Kot ime naj bo navedeno samo ime datoteke, brez morebitnih dodatnih poti do nje.

Funkcija naj omenjen slovar vrne.

(V nadaljevanju bi omenjen slovar uporabili, da odvečne datoteke pobrišete, hkrati pa si zapomnete, katero vsebino morate pod določenim imenom tudi vrniti.)

In [ ]:
import hashlib
from pathlib import Path

# ../data/ sem zamenjal z moj_path saj so se mi datoteke rahlo premaknile.

def deduplication(filepath: str) -> dict[str, set[str]]:
    hash_dict = {}
    folder = Path(filepath)

    for file in folder.glob("*.bin"):
        with open(file, "rb") as f:
            content = f.read()

        file_hash = hashlib.sha256(content).digest()

        if file_hash not in hash_dict:
            hash_dict[file_hash] = set()
        hash_dict[file_hash].add(file.name)

    return hash_dict



deduplication("moj_path")


{b'\x0f\xe6\x86xgR\xb6\t\x0e\xdd\x8c\x9a^\xa7\x1a%]I\x9a\x13X\n\x06m@\xb9\\-\xc1\x08\x85\xb4': {'abusiveness-spines-hitter-wakes.bin',
  'bicameral-courting-cache-colleagues.bin',
  'breadfruits-capitulations-thorough-platefuls.bin',
  'predisposition-aristophanes-billy-breakpoints.bin',
  'relentlessly-sexting-roils-rpm.bin',
  'steels-ridgepole-nate-publisher.bin',
  'undercoated-thee-wounds-upholsters.bin',
  'weeding-threnody-renal-paunchiest.bin'},
 b'b\x95\xae\x19\x08\x9d\xa3g"A\x81\xad\xc3O H!Z\xe6\xdbM+\x0c\xbaUx\xce\xd0\x07\x8fk\xe2': {'amado-turnovers-disfigurements-carton.bin',
  'ashen-mistiness-grouch-underpins.bin',
  'chap-summary-anthropological-arkhangelsk.bin',
  'dependencies-rocked-gofer-animation.bin',
  'junketing-peppercorn-orangutangs-individualism.bin',
  'klaus-liberia-recollected-stop.bin',
  'pete-futilely-idiomatically-existentialists.bin',
  'prioritizes-motliest-daybreak-circumcision.bin',
  'subcultures-extolling-aides-anesthesiology.bin',
  'waterboard-

In [ ]:
def test_deduplication():
    import pickle
    
    # ../data/assignment7.pkl in ../data/ sem zamenjal z moj path saj so se mi datoteke rahlo premaknile.
    
    with open("moj_path", "rb") as h:
        expected = pickle.load(h)

    actual = deduplication("moj_path")
    

    assert len(actual) == len(expected)
    
    for k, v in expected.items():
        if v != {'msg.bin'}: # iz primerjanja izpustimo datoteko msg.bin
            assert actual[k] == v
    
test_deduplication()